In [78]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import math
from sklearn.linear_model import SGDClassifier
import pylab as pl
import random
import pdb
import sklearn as sk
import scipy
from scipy.spatial.distance import pdist, squareform, cdist
from cvxopt import matrix, solvers
from sklearn import svm
from __future__ import division
import pandas as pd

  Implement the dual form of linear SVMs with slack variables. Do not use the built-in SVM implementa-tion in Matlab or sklearn. Instead, write a program that takes data as input, converts it to the appropriate objective function and constraints, and then calls a quadratic programming package to solve it. See thefileoptimizers.txtfor installation and usage for matlab/python.Show in your report the constraints and objective that you generate for the 2D problem with positiveexamples (2, 2), (2, 3) and negative examples (0, -1), (-3, -2). Which examples are support vectors?Note:When using quadratic program solvers, be careful when computing support vectors based on thesolution.  Optimizers do not always drive variables to 0 despite them being 0 in the optimal solution.Thus you may need to perform thresholding onαvalues to extract the correct support vectors.   Wesuggest using a threshold of around10−4, but please check yourαvalues to ensure you are correctlycounting the number of support vectors!

In [2]:
def getData(name):
    data = pl.loadtxt('/Users/becca/GitHub/6.867/code_hw2/data/' + name)
    # Returns column matrices
    X = data.T[0:2]
    Y = data.T[2:3]
    return X, Y 
    
def data(dataset, category):
    if dataset == 'data2.1':
        X = (np.array([[2, 2], [2, 3], [0, -1], [-3, -2]])).T
        Xt = X.T
        Y = (np.array([1, 1, -1, -1])).T[np.newaxis]
        Yt = Y.T
        return X, Y
    else:
        return getData('data'+str(dataset)+'_'+category+'.csv')

In [403]:
# testing
X, Y = data('data2.1', '')

array([[ 2,  2,  0, -3],
       [ 2,  3, -1, -2]])

In [ ]:
########### Define functions

In [423]:
alpha, C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, 1, 1e-4)

     pcost       dcost       gap    pres   dres
 0: -3.2681e-01 -5.4491e+00  2e+01  2e+00  5e-16
 1: -1.5740e-01 -2.4179e+00  3e+00  1e-01  2e-16
 2: -1.1777e-01 -3.4786e-01  2e-01  5e-03  2e-15
 3: -1.4761e-01 -1.6162e-01  1e-02  5e-05  2e-16
 4: -1.5377e-01 -1.5393e-01  2e-04  6e-07  2e-16
 5: -1.5385e-01 -1.5385e-01  2e-06  6e-09  2e-16
 6: -1.5385e-01 -1.5385e-01  2e-08  6e-11  2e-16
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


In [430]:
P

NameError: name 'P' is not defined

In [ ]:
Xt = X.T
Yt = Y.T
# Data range
a = min(X[0])-0.5
b = max(X[0])+0.5
c = min(X[1])-0.5
d = max(X[1])+0.5

# matrices for solver
A = matrix(Yt.T, tc = 'd')
b_mat = matrix(0, tc = 'd')
G = matrix(np.vstack(((-1*np.identity(len(Yt))), np.identity(len(Yt)))))
h = matrix(np.hstack((np.zeros(len(Yt)), np.repeat(C, len(Yt))))[np.newaxis].T)
q = matrix(np.repeat(-1, len(Yt))[np.newaxis].T, tc = 'd')
K = Yt*Xt
P = matrix(np.dot(K, K.T), tc = 'd')

[100.0, 100.0, 100.0, 100.0, 100.0]

In [410]:
Yt = Y.T
q = np.repeat(-1, len(Yt))[np.newaxis].T
q

array([[-1],
       [-1],
       [-1],
       [-1]])

In [422]:
def svmlin(X, Y, C, tol):
    Xt = X.T
    Yt = Y.T
    # Data range
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    
    # matrices for solver
    A = matrix(Yt.T, tc = 'd')
    b_mat = matrix(0, tc = 'd')
    G = matrix(np.vstack(((-1*np.identity(len(Yt))), np.identity(len(Yt)))))
    h = matrix(np.hstack((np.zeros(len(Yt)), np.repeat(C, len(Yt))))[np.newaxis].T)
    q = matrix(np.repeat(-1, len(Yt))[np.newaxis].T, tc = 'd')
    K = Yt*Xt
    P = matrix(np.dot(K, K.T), tc = 'd')
    # Call solver
    sol = solvers.qp(P,q,G,h,A,b_mat)
    alpha = np.asarray((sol['x']))
    status = sol['status']
    # Threshold alpha
    tol = tol
    alpha[alpha < tol] = 0
    theta = np.dot(X, np.multiply(alpha, Yt))
    
    cond = (alpha > tol).reshape(-1)
    SV = Xt[cond] #support vectors
    bias_vec = Yt[cond] - np.dot(Xt[cond], theta)
    test=np.histogram(bias_vec, bins=50000)
    max(test[0])
    bias = np.mean(test[1][test[0]==max(test[0])]) #most frequent bias
    bias
    return C, SV, theta, bias, a, b, c, d, bias_vec

In [251]:
def svmgauss(X, Y, C, gamma, tol):
    Xt = X.T
    Yt = Y.T
    # Data range
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    
    # matrices for solver
    A = matrix(Yt.T, tc = 'd')
    b_mat = matrix(0, tc = 'd')
    G = matrix(np.vstack(((-1*np.identity(len(Yt))), np.identity(len(Yt)))))
    h = matrix(np.hstack((np.zeros(len(Yt)), np.repeat(C, len(Yt))))[np.newaxis].T)
    q = matrix(np.repeat(-1, len(Yt))[np.newaxis].T, tc = 'd')
    pairwise_sq_dists = squareform(pdist(Xt, 'sqeuclidean'))
    K = Yt*Y*scipy.exp(-gamma*pairwise_sq_dists)
    P = matrix(K, tc = 'd')
    
    # Call solver
    sol = solvers.qp(P,q,G,h,A,b_mat)
    alpha = np.asarray((sol['x']))
    status = sol['status']
    # Threshold alpha
    tol = tol
    alpha[alpha < tol] = 0
    theta = np.dot(scipy.exp(-gamma*pairwise_sq_dists), np.multiply(alpha, Yt))
    cond = (alpha > tol).reshape(-1)
    cond2 = ((alpha < C) > tol).reshape(-1)
    Xt_sv = Xt[cond]
    sq_dists = (cdist(Xt_sv, Xt_sv, 'sqeuclidean'))
    Z = np.dot(scipy.exp(-gamma*sq_dists), np.multiply(alpha[cond], Yt[cond]))    
    SV = Xt[cond] #support vectors
    bias_vec = Yt[cond]-Z    
    test=np.histogram(bias_vec, bins=50000)
    max(test[0])
    bias = np.mean(test[1][test[0]==max(test[0])]) #most frequent bias
    bias
    return cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec

In [7]:
# Plot SVM decision boundary - my linear implementation
def svmlinplot(X, Y, C, theta, bias, a, b, c, d, name):
    fig = plt.figure(figsize=(5,5), dpi=300)
    delta = 0.01
    xx, yy = np.meshgrid(np.arange(a, b, delta), np.arange(c, d, delta))
    Z = np.dot(theta.T, np.vstack((xx.ravel(), yy.ravel()))) + bias
    Z[Z > 0] = 1
    Z[Z < 0] = -1
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    plt.scatter(X[0], X[1], c = Y, edgecolors = 'k', cmap = plt.cm.Paired) 

    plt.xlabel('X1')
    plt.ylabel('X2')
    fig.savefig('svmlinear_%s_C=%s.jpg' %(name, C) , bbox_inches='tight')

In [8]:
# Plot SVM decision boundary - my linear implementation
def svmgaussplot(X, Y, Yfit, cond, C, gamma, alpha, SV, bias, a, b, c, d, name):
    plt.rcParams.update({'font.size': 15})
    fig = plt.figure(figsize=(5,5), dpi=300)
    Yt = Yfit.T
    delta = 0.01
    xx, yy = np.meshgrid(np.arange(a, b, delta), np.arange(c, d, delta))
    xxyy = np.vstack((xx.ravel(), yy.ravel()))
    Xt_sv = SV
    sq_dists = (cdist(xxyy.T, Xt_sv, 'sqeuclidean'))
    Z = np.dot(scipy.exp(-gamma*sq_dists), np.multiply(alpha[cond], Yt[cond])) + bias 
    Z[Z > 0] = 1
    Z[Z < 0] = -1
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    plt.scatter(X[0], X[1], c = Y, edgecolors = 'k', cmap = plt.cm.Paired) 
    
    plt.xlabel('X1')
    plt.ylabel('X2')
    fig.savefig('svmGauss_%s_C=%s_gamma=%s.jpg' %(name, C, gamma) , bbox_inches='tight')

In [18]:
def sklearngauss(X, Y, C, gamma):
    Xt = X.T
    Yt = Y.T
    model = svm.SVC(kernel='rbf', C = C, gamma = gamma) 
    model.fit(X.T, np.ndarray.flatten(Yt))
    predicted = model.predict(Xt)
    intercept = model.intercept_
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    return predicted, intercept, model, a, b, c, d

In [15]:
from sklearn import svm
def sklearnlin(X, Y, C):
    Xt = X.T
    Yt = Y.T
    model = svm.SVC(kernel='linear', C = C) 
    model.fit(X.T, np.ndarray.flatten(Yt))
    predicted = model.predict(Xt)
    intercept = model.intercept_
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    return predicted, intercept, model, a, b, c, d

In [16]:
# Plot SVM decision boundary for scikit learn
def sklearnplot(X, Y, model, a, b, c, d, name):
    plt.rcParams.update({'font.size': 15})
    fig = plt.figure(figsize=(5,5), dpi=300)
    delta = 0.01
    xx, yy = np.meshgrid(np.arange(a, b, delta), np.arange(c, d, delta))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    plt.scatter(X[0], X[1], c = Y, edgecolors = 'k', cmap = plt.cm.Paired) 
    plt.xlabel('X1')
    plt.ylabel('X2')
    fig.savefig('SK_%s_%s_C=%s_gamma=%s.jpg' %(model.kernel, name, model.C, model.gamma) , bbox_inches='tight')

In [ ]:
#### Problem

In [1030]:
# 2.1

In [19]:
# Use sklearn functions to fit and plot
dset = 'data2.1'
X, Y = data(dset,'')
predicted, intercept, model, a, b, c, d = sklearnlin(X, Y, 1)
sklearnplot(X, Y, model,a,b,c,d, dset)

predicted, intercept, model, a, b, c, d = sklearngauss(X, Y, 1, 1)
sklearnplot(X, Y, model,a,b,c,d, dset)

In [20]:
# Use my functions to fit and plot
dset = 'data2.1'
X, Y = data(dset,'')
#lin
C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, 1, 1e-4)
svmlinplot(X, Y, C, theta, bias, a, b, c, d, dset)
#gauss
cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, 1, 1, 1e-4)
svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset)

     pcost       dcost       gap    pres   dres
 0: -3.2681e-01 -5.4491e+00  2e+01  2e+00  5e-16
 1: -1.5740e-01 -2.4179e+00  3e+00  1e-01  2e-16
 2: -1.1777e-01 -3.4786e-01  2e-01  5e-03  2e-15
 3: -1.4761e-01 -1.6162e-01  1e-02  5e-05  2e-16
 4: -1.5377e-01 -1.5393e-01  2e-04  6e-07  2e-16
 5: -1.5385e-01 -1.5385e-01  2e-06  6e-09  2e-16
 6: -1.5385e-01 -1.5385e-01  2e-08  6e-11  2e-16
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


     pcost       dcost       gap    pres   dres
 0: -1.5783e+00 -5.9595e+00  4e+00  1e-16  3e-16
 1: -1.6381e+00 -1.8886e+00  3e-01  1e-16  2e-16
 2: -1.6874e+00 -1.7144e+00  3e-02  2e-16  1e-16
 3: -1.6892e+00 -1.6900e+00  8e-04  1e-16  4e-17
 4: -1.6892e+00 -1.6893e+00  8e-06  2e-16  1e-16
 5: -1.6892e+00 -1.6892e+00  8e-08  1e-16  4e-17
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


Note for 2.1: for C = 1, array([[ 2,  2],
       [ 0, -1]]) are linear support vectors, for Gaussian all points are support vectors. Class accuracy 100%

In [ ]:
# 2.2

Test your implementation on the 2D datasets.   SetC=1 and report/explain your decision boundary and  classification  error  rate  on  the  training  and  validation  sets.   We  provide  some  skeleton  code  insvmtest.py/mto get you started.

In [21]:
# Define functions to calculate prediction accuracy
def svmgausspred(X, Y, Yfit, SV, cond, gamma, alpha, bias):
    Yt = Yfit.T
    Xt_sv = SV
    sq_dists = (cdist(X.T, Xt_sv, 'sqeuclidean'))
    pred = np.dot(scipy.exp(-gamma*sq_dists), np.multiply(alpha[cond], Yt[cond])) + bias 
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    pred
    accuracy = sum(sum(pred.T == Y))/len(Y[0])*100
    return accuracy

def svmlinpred(X, Y, SV, theta, bias):
    Xt_sv = SV
    pred = np.dot(theta.T, X) + bias
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    pred
    accuracy = sum(sum(pred == Y))/len(Y[0])*100
    return accuracy

In [ ]:
# # Use my functions to fit and plot training and validation - data1
# dset = 'data1train'
# dset2 = 'data1val'
# X, Y = data(1,'train')
# X2, Y2 = data(1,'validate')

# #lin
# C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, 1, 1e-4)
# svmlinplot(X, Y, C, theta, bias, a, b, c, d, dset)
# svmlinplot(X2, Y2, C, theta, bias, a, b, c, d, dset2)

# linacc_train = svmlinpred(X, Y, SV, theta, bias) 
# linacc_val = svmlinpred(X2, Y2, SV, theta, bias)

In [1172]:
#gauss
cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, 1, 1, 1e-4)
svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset)
svmgaussplot(X2, Y2, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset2)

     pcost       dcost       gap    pres   dres
 0: -5.0166e+00 -4.5855e+02  2e+03  1e+00  7e-16
 1:  1.2009e+00 -1.6226e+02  2e+02  2e-02  7e-16
 2: -7.0081e+00 -2.2040e+01  2e+01  1e-03  9e-16
 3: -8.5142e+00 -1.4026e+01  6e+00  4e-04  6e-16
 4: -9.2197e+00 -1.1200e+01  2e+00  7e-05  5e-16
 5: -9.5040e+00 -1.0417e+01  9e-01  1e-05  5e-16
 6: -9.6607e+00 -9.8723e+00  2e-01  2e-06  4e-16
 7: -9.7021e+00 -9.7765e+00  7e-02  7e-16  5e-16
 8: -9.7184e+00 -9.7414e+00  2e-02  4e-16  5e-16
 9: -9.7244e+00 -9.7309e+00  7e-03  3e-16  5e-16
10: -9.7265e+00 -9.7277e+00  1e-03  8e-16  5e-16
11: -9.7270e+00 -9.7270e+00  3e-05  2e-15  5e-16
12: -9.7270e+00 -9.7270e+00  3e-07  2e-15  5e-16
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


In [1195]:
gaussacc_train = svmgausspred(X, Y, Y, SV, cond, gamma, alpha, bias)
gaussacc_val = svmgausspred(X2, Y2, Y, SV, cond, gamma, alpha, bias)

NOTE: 2.2: data 1 both train and val lin+ gauss 100% accuracy (although Gauss not necessary for this part)

In [ ]:
# 2.3

In [22]:
# Use my functions to fit and plot training and validation - data2
dset = 'data2train'
dset2 = 'data2val'
X, Y = data(2,'train')
X2, Y2 = data(2,'validate')

#lin
linacc_train2 = []
linacc_val2 = []
SVnum2lin = []
for i in [.01, .1, 1, 10, 100]:
    C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, i, 1e-4)
    svmlinplot(X, Y, C, theta, bias, a, b, c, d, dset)
    svmlinplot(X2, Y2, C, theta, bias, a, b, c, d, dset2)
    linacc_train2.append(svmlinpred(X, Y, SV, theta, bias))
    linacc_val2.append(svmlinpred(X2, Y2, SV, theta, bias))
    SVnum2lin.append(SV.shape[0])

#gauss
gaussacc_train2 = []
gaussacc_val2 = []
SVnum2gauss = []
for i in [.01, .1, 1, 10, 100]:
    for j in [.01, .1, 1, 10, 100]:
        cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, i, j, 1e-4)
        svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset)
        svmgaussplot(X2, Y2, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset2)
        gaussacc_train2.append(svmgausspred(X, Y, Y, SV, cond, gamma, alpha, bias))
        gaussacc_val2.append(svmgausspred(X2, Y2, Y, SV, cond, gamma, alpha, bias))
        SVnum2gauss.append(SV.shape[0])

     pcost       dcost       gap    pres   dres
 0: -1.1173e+02 -9.6455e+00  3e+03  6e+01  9e-15
 1: -2.9994e+00 -9.5983e+00  5e+01  7e-01  8e-15
 2: -1.7193e+00 -7.1104e+00  7e+00  4e-02  1e-15
 3: -1.8459e+00 -2.4834e+00  7e-01  3e-03  2e-15
 4: -2.1106e+00 -2.2331e+00  1e-01  4e-04  7e-16
 5: -2.1631e+00 -2.1915e+00  3e-02  7e-05  7e-16
 6: -2.1733e+00 -2.1824e+00  9e-03  2e-05  7e-16
 7: -2.1772e+00 -2.1788e+00  2e-03  2e-06  8e-16
 8: -2.1779e+00 -2.1782e+00  3e-04  4e-07  9e-16
 9: -2.1780e+00 -2.1780e+00  1e-05  8e-09  9e-16
10: -2.1780e+00 -2.1780e+00  1e-06  5e-10  9e-16
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


     pcost       dcost       gap    pres   dres
 0: -1.2168e+02 -9.6789e+01  4e+03  3e+01  1e-14
 1: -1.9667e+01 -9.1702e+01  2e+02  1e+00  8e-15
 2: -1.4498e+01 -4.7662e+01  4e+01  1e-01  2e-15
 3: -1.5653e+01 -2.0439e+01  5e+00  1e-02  4e-15
 4: -1.6970e+01 -1.8749e+01  2e+00  4e-03  2e-15
 5: -1.7300e+01 -1.8332e+01  1e+00  2e-03  1e-15
 6: -1.7574e+01 -1.7979e+01  4e-01  6e-04  1e-15
 7: -1.7693e+01 -1.7828e+01  1e-01  2e-04  1e-15
 8: -1.7737e+01 -1.7772e+01  4e-02  2e-05  1e-15
 9: -1.7752e+01 -1.7755e+01  3e-03  1e-06  2e-15
10: -1.7754e+01 -1.7754e+01  3e-05  1e-08  2e-15
11: -1.7754e+01 -1.7754e+01  3e-07  1e-10  2e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.2102e+02 -1.1032e+03  6e+03  3e+00  2e-14
 1: -1.4926e+02 -7.6319e+02  1e+03  4e-01  1e-14
 2: -1.3704e+02 -2.4046e+02  1e+02  1e-02  1e-14
 3: -1.5807e+02 -1.8764e+02  3e+01  3e-03  1e-14
 4: -1.6226e+02 -1.8229e+02  2e+01  2e-03  1e-14
 5: -1.6618e+02 -1.7648e+02  1e+01  7e-04  1e-1

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:36: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


     pcost       dcost       gap    pres   dres
 0: -8.6852e+01 -1.2820e+01  3e+03  6e+01  2e-15
 1: -7.8898e+00 -1.0746e+01  1e+02  2e+00  2e-15
 2: -2.8847e+00 -9.1570e+00  7e+00  2e-02  1e-15
 3: -3.0673e+00 -4.0757e+00  1e+00  2e-03  9e-16
 4: -3.2982e+00 -3.4236e+00  1e-01  3e-04  8e-16
 5: -3.3091e+00 -3.3775e+00  7e-02  9e-05  6e-16
 6: -3.3187e+00 -3.3628e+00  4e-02  4e-05  5e-16
 7: -3.3289e+00 -3.3477e+00  2e-02  1e-05  6e-16
 8: -3.3351e+00 -3.3391e+00  4e-03  2e-06  8e-16
 9: -3.3366e+00 -3.3371e+00  5e-04  4e-17  8e-16
10: -3.3368e+00 -3.3369e+00  6e-06  2e-16  9e-16
11: -3.3369e+00 -3.3369e+00  6e-08  2e-16  8e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -5.2138e+01 -1.5183e+01  3e+03  5e+01  8e-16
 1: -1.0496e+01 -1.1947e+01  2e+02  3e+00  1e-15
 2: -3.6474e+00 -9.4042e+00  1e+01  1e-01  1e-15
 3: -3.1599e+00 -4.4759e+00  1e+00  6e-04  2e-15
 4: -3.4127e+00 -3.5460e+00  1e-01  5e-05  9e-16
 5: -3.4215e+00 -3.4806e+00  6e-02  2e-05  6e-1

/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


     pcost       dcost       gap    pres   dres
 0: -5.0143e+01 -2.3347e+01  3e+03  5e+01  6e-16
 1: -1.9206e+01 -1.3998e+01  1e+02  3e+00  6e-16
 2: -3.6631e+00 -9.2471e+00  8e+00  4e-02  2e-15
 3: -3.5964e+00 -4.6079e+00  1e+00  6e-03  6e-16
 4: -3.7636e+00 -3.8976e+00  1e-01  7e-04  5e-16
 5: -3.7601e+00 -3.7836e+00  3e-02  1e-04  8e-16
 6: -3.7603e+00 -3.7816e+00  2e-02  7e-05  1e-15
 7: -3.7622e+00 -3.7680e+00  6e-03  1e-05  4e-16
 8: -3.7626e+00 -3.7671e+00  5e-03  6e-06  4e-16
 9: -3.7637e+00 -3.7649e+00  1e-03  1e-06  5e-16
10: -3.7640e+00 -3.7643e+00  3e-04  1e-16  6e-16
11: -3.7641e+00 -3.7642e+00  6e-05  1e-16  6e-16
12: -3.7642e+00 -3.7642e+00  2e-06  2e-16  5e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -8.6665e+01 -3.0931e+01  2e+03  5e+01  3e-16
 1: -2.5455e+01 -1.0550e+01  1e+02  3e+00  5e-16
 2: -3.4190e+00 -8.4060e+00  5e+00  5e-16  1e-15
 3: -3.7435e+00 -4.2230e+00  5e-01  5e-17  5e-16
 4: -3.8618e+00 -3.9229e+00  6e-02  1e-16  3e-1

     pcost       dcost       gap    pres   dres
 0: -5.1764e+02 -1.8700e+04  3e+04  3e-01  2e-14
 1: -5.3921e+02 -3.6367e+03  3e+03  2e-02  2e-14
 2: -7.9310e+02 -1.9823e+03  1e+03  6e-03  2e-14
 3: -9.2307e+02 -1.6808e+03  8e+02  3e-03  2e-14
 4: -1.0111e+03 -1.3917e+03  4e+02  1e-03  2e-14
 5: -1.0497e+03 -1.3232e+03  3e+02  7e-04  2e-14
 6: -1.0865e+03 -1.2355e+03  2e+02  3e-04  3e-14
 7: -1.1127e+03 -1.1862e+03  7e+01  1e-04  3e-14
 8: -1.1270e+03 -1.1604e+03  3e+01  4e-05  3e-14
 9: -1.1324e+03 -1.1486e+03  2e+01  3e-06  3e-14
10: -1.1371e+03 -1.1423e+03  5e+00  7e-07  3e-14
11: -1.1384e+03 -1.1406e+03  2e+00  2e-07  3e-14
12: -1.1392e+03 -1.1397e+03  4e-01  3e-08  3e-14
13: -1.1394e+03 -1.1395e+03  5e-02  8e-15  4e-14
14: -1.1394e+03 -1.1394e+03  5e-04  2e-14  4e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.8244e+02 -1.5175e+04  3e+04  3e-01  8e-15
 1:  3.6889e+01 -2.5440e+03  3e+03  2e-02  7e-15
 2: -3.1772e+02 -1.0370e+03  7e+02  3e-03  6e-1

In [401]:
X, Y = data('data 2.1','')

C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, i, 1e-4)
SV

IOError: [Errno 2] No such file or directory: '/Users/becca/GitHub/6.867/code_hw2/data/datadata 2.1_.csv'

In [ ]:
# Use my functions to fit and plot training and validation - data3
dset = 'data3train'
dset2 = 'data3val'
X, Y = data(3,'train')
X2, Y2 = data(3,'validate')

#lin
linacc_train3 = []
linacc_val3 = []
SVnum3lin = []
for i in [.01, .1, 1, 10, 100]:
    C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, i, 1e-4)
    svmlinplot(X, Y, C, theta, bias, a, b, c, d, dset)
    svmlinplot(X2, Y2, C, theta, bias, a, b, c, d, dset2)
    linacc_train3.append(svmlinpred(X, Y, SV, theta, bias))
    linacc_val3.append(svmlinpred(X2, Y2, SV, theta, bias))
    SVnum3lin.append(SV.shape[0])

#gauss
gaussacc_train3 = []
gaussacc_val3 = []
SVnum3gauss = []
for i in [.01, .1, 1, 10, 100]:
    for j in [.01, .1, 1, 10, 100]:
        cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, i, j, 1e-4)
        svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset)
        svmgaussplot(X2, Y2, Y, cond, C, gamma, alpha, SV, bias, a, b, c, d, dset2)
        gaussacc_train3.append(svmgausspred(X, Y, Y, SV, cond, gamma, alpha, bias))
        gaussacc_val3.append(svmgausspred(X2, Y2, Y, SV, cond, gamma, alpha, bias))
        SVnum3gauss.append(SV.shape[0])

In [ ]:
# Use my functions to fit and plot training and validation - data4
dset = 'data4train'
dset2 = 'data4val'
X, Y = data(4,'train')
X2, Y2 = data(4,'validate')

#lin
linacc_train4 = []
linacc_val4 = []
SVnum4lin = []
for i in [.01, .1, 1, 10, 100]:
    C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, i, 1e-4)
    svmlinplot(X, Y, C, theta, bias, a-1, b, c, d, dset)
    svmlinplot(X2, Y2, C, theta, bias, a-1, b, c, d, dset2)
    linacc_train4.append(svmlinpred(X, Y, SV, theta, bias))
    linacc_val4.append(svmlinpred(X2, Y2, SV, theta, bias))
    SVnum4lin.append(SV.shape[0])

#gauss
gaussacc_train4 = []
gaussacc_val4 = []
SVnum4gauss = []
for i in [.01, .1, 1, 10, 100]:
    for j in [.01, .1, 1, 10, 100]:
        cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, i, j, 1e-4)
        svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a-1, b, c, d, dset)
        svmgaussplot(X2, Y2, Y, cond, C, gamma, alpha, SV, bias, a-1, b, c, d, dset2)
        gaussacc_train4.append(svmgausspred(X, Y, Y, SV, cond, gamma, alpha, bias))
        gaussacc_val4.append(svmgausspred(X2, Y2, Y, SV, cond, gamma, alpha, bias))
        SVnum4gauss.append(SV.shape[0])

In [ ]:
# Use my functions to fit and plot training and validation - data1
dset = 'data1train'
dset2 = 'data1val'
X, Y = data(1,'train')
X2, Y2 = data(1,'validate')

#lin
linacc_train1 = []
linacc_val1 = []
SVnum1lin = []
for i in [.01, .1, 1, 10, 100]:
    C, SV, theta, bias, a, b, c, d, bias_vec = svmlin(X, Y, i, 1e-4)
    svmlinplot(X, Y, C, theta, bias, a-1, b, c, d, dset)
    svmlinplot(X2, Y2, C, theta, bias, a-1, b, c, d, dset2)
    linacc_train1.append(svmlinpred(X, Y, SV, theta, bias))
    linacc_val1.append(svmlinpred(X2, Y2, SV, theta, bias))
    SVnum1lin.append(SV.shape[0])

#gauss
gaussacc_train1 = []
gaussacc_val1 = []
SVnum1gauss = []
for i in [.01, .1, 1, 10, 100]:
    for j in [.01, .1, 1, 10, 100]:
        cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec = svmgauss(X, Y, i, j, 1e-4)
        svmgaussplot(X, Y, Y, cond, C, gamma, alpha, SV, bias, a-1, b, c, d, dset)
        svmgaussplot(X2, Y2, Y, cond, C, gamma, alpha, SV, bias, a-1, b, c, d, dset2)
        gaussacc_train1.append(svmgausspred(X, Y, Y, SV, cond, gamma, alpha, bias))
        gaussacc_val1.append(svmgausspred(X2, Y2, Y, SV, cond, gamma, alpha, bias))
        SVnum1gauss.append(SV.shape[0])

In [457]:
# Generate DFs for plotting
C = np.repeat([.01, .1, 1, 10, 100], 5)
gamma = np.tile([.01, .1, 1, 10, 100], 5)
C2 = [.01, .1, 1, 10, 100]

gauss1sv = pd.DataFrame(data = np.array([C, gamma, SVnum1gauss]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss2sv = pd.DataFrame(data = np.array([C, gamma, SVnum2gauss]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss3sv = pd.DataFrame(data = np.array([C, gamma, SVnum3gauss]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss4sv = pd.DataFrame(data = np.array([C, gamma, SVnum4gauss]).T, columns = ['C', r'$\gamma$', 'SV Number'])

linsv = pd.DataFrame(data = np.array([SVnum1lin, SVnum2lin, SVnum3lin, SVnum4lin]).T, 
                     columns = ['Data 1', 'Data 2', 'Data 3', 'Data 4'])
linsv.index = C2

linacc = pd.DataFrame(data = np.array([linacc_train1, linacc_val1, linacc_train2, linacc_val2, 
                                       linacc_train3, linacc_val3, linacc_train4, linacc_val4]).T, 
                     columns = ['D1 Train', 'D1 Validate', 'D2 Train', 'D2 Validate', 'D3 Train', 'D3 Validate',
                                'D4 Train', 'D4 Validate'])
linacc.index = C2

linaccC1 = pd.DataFrame(data = np.array([linacc_train1[2], linacc_val1[2], linacc_train2[2], linacc_val2[2], 
                                       linacc_train3[2], linacc_val3[2], linacc_train4[2], linacc_val4[2]]).T, columns = [''])
linaccC1.index = ['D1 Train', 'D1 Validate', 'D2 Train', 'D2 Validate', 'D3 Train', 'D3 Validate',
                                'D4 Train', 'D4 Validate']

gauss1acctr = pd.DataFrame(data = np.array([C, gamma, gaussacc_train1]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss2acctr = pd.DataFrame(data = np.array([C, gamma, gaussacc_train2]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss3acctr = pd.DataFrame(data = np.array([C, gamma, gaussacc_train3]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss4acctr = pd.DataFrame(data = np.array([C, gamma, gaussacc_train4]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss1accval = pd.DataFrame(data = np.array([C, gamma, gaussacc_val1]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss2accval = pd.DataFrame(data = np.array([C, gamma, gaussacc_val2]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss3accval = pd.DataFrame(data = np.array([C, gamma, gaussacc_val3]).T, columns = ['C', r'$\gamma$', 'SV Number'])
gauss4accval = pd.DataFrame(data = np.array([C, gamma, gaussacc_val4]).T, columns = ['C', r'$\gamma$', 'SV Number'])

In [472]:
a=gauss1sv.pivot("C", r'$\gamma$', "SV Number")
a?

In [346]:
# Gauss SV Plot
import seaborn as sns


f,(ax1, ax2, ax3, ax4) = plt.subplots(1,4,sharex = True, sharey = 'row', figsize = (12, 3))

g1 = sns.heatmap(gauss1sv.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax1)
g2 = sns.heatmap(gauss2sv.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax2)
g3 = sns.heatmap(gauss3sv.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax3)
g4 = sns.heatmap(gauss4sv.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax4)
plt.subplots_adjust(wspace = .25)
ax2.set_ylabel('')
ax3.set_ylabel('')
ax4.set_ylabel('')
ax1.set_yticklabels(ax1.get_yticklabels(), rotation = 0)
ax1.set_ylabel(ax1.get_ylabel(), rotation = 0)
ax1.set_title('Data 1')
ax2.set_title('Data 2')
ax3.set_title('Data 3')
ax4.set_title('Data 4')

plt.savefig('Gauss SV number 1-4.jpg', dpi= 300, bbox_inches='tight')

In [ ]:
# Gauss Accuracy Plot
f, [[ax1, ax2, ax3, ax4], [ax5, ax6, ax7, ax8]] = plt.subplots(2,4,sharex = True, sharey = 'row', figsize = (12, 6))

g1 = sns.heatmap(gauss1acctr.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax1, vmin = 50, vmax = 100)
g2 = sns.heatmap(gauss2acctr.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax2, vmin = 50, vmax = 100)
g3 = sns.heatmap(gauss3acctr.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax3, vmin = 50, vmax = 100)
g4 = sns.heatmap(gauss4acctr.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax4, vmin = 50, vmax = 100)
g5 = sns.heatmap(gauss1accval.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax5, vmin = 50, vmax = 100)
g6 = sns.heatmap(gauss2accval.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax6, vmin = 50, vmax = 100)
g7 = sns.heatmap(gauss3accval.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax7, vmin = 50, vmax = 100)
g8 = sns.heatmap(gauss4accval.pivot("C", r'$\gamma$', "SV Number"), cmap = viridis, ax = ax8, vmin = 50, vmax = 100)

plt.subplots_adjust(wspace = .25)
#plt.subplots_adjust(hspace = .25)

ax1.set_xlabel('')
ax2.set_xlabel('')
ax3.set_xlabel('')
ax4.set_xlabel('')

ax2.set_ylabel('')
ax3.set_ylabel('')
ax4.set_ylabel('')
ax4.set_ylabel('')
ax6.set_ylabel('')
ax7.set_ylabel('')
ax8.set_ylabel('')
ax1.set_yticklabels(ax1.get_yticklabels(), rotation = 0)
ax1.set_ylabel(ax1.get_ylabel(), rotation = 0)
ax5.set_ylabel(ax1.get_ylabel(), rotation = 0)
ax1.set_title('Data 1')
ax2.set_title('Data 2')
ax3.set_title('Data 3')
ax4.set_title('Data 4')

plt.text(-27.2, 9, 'Training', rotation = 90)
plt.text(-27.2, 3.4, 'Validation', rotation = 90)

plt.savefig('Gauss Accuracy number 1-4.jpg', dpi= 300, bbox_inches='tight')

In [476]:
# Linear SV plot
sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True)
g1 = sns.heatmap(linsv, cmap = viridis, ax = ax1, annot = True, annot_kws={"size": 15})
plt.rcParams.update({'font.size': 15})
plt.yticks(rotation=0) 
plt.ylabel('C')
plt.savefig('Linear SV number 1-4 v2.jpg', dpi= 300, bbox_inches='tight')

In [475]:
linsv

,Data 1,Data 2,Data 3,Data 4
0.01,75,252,184,399
0.10,20,186,68,393
1.00,4,173,33,392
10.00,3,173,20,392
100.00,3,172,17,393


In [464]:
# Linear Accuracy plot C = 1 only
sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True, figsize = (3,5))
g1 = sns.heatmap(linaccC1, cmap = viridis, ax = ax1, annot = True, annot_kws={"size": 16})
plt.yticks(rotation=0) 
plt.xlabel('')
plt.savefig('Linear Accuracy number 1-4 C = 1 only.jpg', dpi= 300, bbox_inches='tight')

In [467]:
gauss4accval

,C,$\gamma$,SV Number
0,0.01,0.01,50.00
1,0.01,0.10,71.75
2,0.01,1.00,54.25
3,0.01,10.00,50.00
4,0.01,100.00,50.00
5,0.10,0.01,88.25
6,0.10,0.10,93.50
7,0.10,1.00,94.75
8,0.10,10.00,50.00
9,0.10,100.00,50.00


In [264]:
# Linear Accuracy plot
sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True)
g1 = sns.heatmap(linacc, cmap = viridis, ax = ax1, annot = True, annot_kws={"size": 10})
plt.yticks(rotation=0) 
plt.ylabel('C')

plt.savefig('Linear Accuracy number 1-4.jpg', dpi= 300, bbox_inches='tight')

In [68]:
_viridis_data = [[0.267004, 0.004874, 0.329415],
                 [0.268510, 0.009605, 0.335427],
                 [0.269944, 0.014625, 0.341379],
                 [0.271305, 0.019942, 0.347269],
                 [0.272594, 0.025563, 0.353093],
                 [0.273809, 0.031497, 0.358853],
                 [0.274952, 0.037752, 0.364543],
                 [0.276022, 0.044167, 0.370164],
                 [0.277018, 0.050344, 0.375715],
                 [0.277941, 0.056324, 0.381191],
                 [0.278791, 0.062145, 0.386592],
                 [0.279566, 0.067836, 0.391917],
                 [0.280267, 0.073417, 0.397163],
                 [0.280894, 0.078907, 0.402329],
                 [0.281446, 0.084320, 0.407414],
                 [0.281924, 0.089666, 0.412415],
                 [0.282327, 0.094955, 0.417331],
                 [0.282656, 0.100196, 0.422160],
                 [0.282910, 0.105393, 0.426902],
                 [0.283091, 0.110553, 0.431554],
                 [0.283197, 0.115680, 0.436115],
                 [0.283229, 0.120777, 0.440584],
                 [0.283187, 0.125848, 0.444960],
                 [0.283072, 0.130895, 0.449241],
                 [0.282884, 0.135920, 0.453427],
                 [0.282623, 0.140926, 0.457517],
                 [0.282290, 0.145912, 0.461510],
                 [0.281887, 0.150881, 0.465405],
                 [0.281412, 0.155834, 0.469201],
                 [0.280868, 0.160771, 0.472899],
                 [0.280255, 0.165693, 0.476498],
                 [0.279574, 0.170599, 0.479997],
                 [0.278826, 0.175490, 0.483397],
                 [0.278012, 0.180367, 0.486697],
                 [0.277134, 0.185228, 0.489898],
                 [0.276194, 0.190074, 0.493001],
                 [0.275191, 0.194905, 0.496005],
                 [0.274128, 0.199721, 0.498911],
                 [0.273006, 0.204520, 0.501721],
                 [0.271828, 0.209303, 0.504434],
                 [0.270595, 0.214069, 0.507052],
                 [0.269308, 0.218818, 0.509577],
                 [0.267968, 0.223549, 0.512008],
                 [0.266580, 0.228262, 0.514349],
                 [0.265145, 0.232956, 0.516599],
                 [0.263663, 0.237631, 0.518762],
                 [0.262138, 0.242286, 0.520837],
                 [0.260571, 0.246922, 0.522828],
                 [0.258965, 0.251537, 0.524736],
                 [0.257322, 0.256130, 0.526563],
                 [0.255645, 0.260703, 0.528312],
                 [0.253935, 0.265254, 0.529983],
                 [0.252194, 0.269783, 0.531579],
                 [0.250425, 0.274290, 0.533103],
                 [0.248629, 0.278775, 0.534556],
                 [0.246811, 0.283237, 0.535941],
                 [0.244972, 0.287675, 0.537260],
                 [0.243113, 0.292092, 0.538516],
                 [0.241237, 0.296485, 0.539709],
                 [0.239346, 0.300855, 0.540844],
                 [0.237441, 0.305202, 0.541921],
                 [0.235526, 0.309527, 0.542944],
                 [0.233603, 0.313828, 0.543914],
                 [0.231674, 0.318106, 0.544834],
                 [0.229739, 0.322361, 0.545706],
                 [0.227802, 0.326594, 0.546532],
                 [0.225863, 0.330805, 0.547314],
                 [0.223925, 0.334994, 0.548053],
                 [0.221989, 0.339161, 0.548752],
                 [0.220057, 0.343307, 0.549413],
                 [0.218130, 0.347432, 0.550038],
                 [0.216210, 0.351535, 0.550627],
                 [0.214298, 0.355619, 0.551184],
                 [0.212395, 0.359683, 0.551710],
                 [0.210503, 0.363727, 0.552206],
                 [0.208623, 0.367752, 0.552675],
                 [0.206756, 0.371758, 0.553117],
                 [0.204903, 0.375746, 0.553533],
                 [0.203063, 0.379716, 0.553925],
                 [0.201239, 0.383670, 0.554294],
                 [0.199430, 0.387607, 0.554642],
                 [0.197636, 0.391528, 0.554969],
                 [0.195860, 0.395433, 0.555276],
                 [0.194100, 0.399323, 0.555565],
                 [0.192357, 0.403199, 0.555836],
                 [0.190631, 0.407061, 0.556089],
                 [0.188923, 0.410910, 0.556326],
                 [0.187231, 0.414746, 0.556547],
                 [0.185556, 0.418570, 0.556753],
                 [0.183898, 0.422383, 0.556944],
                 [0.182256, 0.426184, 0.557120],
                 [0.180629, 0.429975, 0.557282],
                 [0.179019, 0.433756, 0.557430],
                 [0.177423, 0.437527, 0.557565],
                 [0.175841, 0.441290, 0.557685],
                 [0.174274, 0.445044, 0.557792],
                 [0.172719, 0.448791, 0.557885],
                 [0.171176, 0.452530, 0.557965],
                 [0.169646, 0.456262, 0.558030],
                 [0.168126, 0.459988, 0.558082],
                 [0.166617, 0.463708, 0.558119],
                 [0.165117, 0.467423, 0.558141],
                 [0.163625, 0.471133, 0.558148],
                 [0.162142, 0.474838, 0.558140],
                 [0.160665, 0.478540, 0.558115],
                 [0.159194, 0.482237, 0.558073],
                 [0.157729, 0.485932, 0.558013],
                 [0.156270, 0.489624, 0.557936],
                 [0.154815, 0.493313, 0.557840],
                 [0.153364, 0.497000, 0.557724],
                 [0.151918, 0.500685, 0.557587],
                 [0.150476, 0.504369, 0.557430],
                 [0.149039, 0.508051, 0.557250],
                 [0.147607, 0.511733, 0.557049],
                 [0.146180, 0.515413, 0.556823],
                 [0.144759, 0.519093, 0.556572],
                 [0.143343, 0.522773, 0.556295],
                 [0.141935, 0.526453, 0.555991],
                 [0.140536, 0.530132, 0.555659],
                 [0.139147, 0.533812, 0.555298],
                 [0.137770, 0.537492, 0.554906],
                 [0.136408, 0.541173, 0.554483],
                 [0.135066, 0.544853, 0.554029],
                 [0.133743, 0.548535, 0.553541],
                 [0.132444, 0.552216, 0.553018],
                 [0.131172, 0.555899, 0.552459],
                 [0.129933, 0.559582, 0.551864],
                 [0.128729, 0.563265, 0.551229],
                 [0.127568, 0.566949, 0.550556],
                 [0.126453, 0.570633, 0.549841],
                 [0.125394, 0.574318, 0.549086],
                 [0.124395, 0.578002, 0.548287],
                 [0.123463, 0.581687, 0.547445],
                 [0.122606, 0.585371, 0.546557],
                 [0.121831, 0.589055, 0.545623],
                 [0.121148, 0.592739, 0.544641],
                 [0.120565, 0.596422, 0.543611],
                 [0.120092, 0.600104, 0.542530],
                 [0.119738, 0.603785, 0.541400],
                 [0.119512, 0.607464, 0.540218],
                 [0.119423, 0.611141, 0.538982],
                 [0.119483, 0.614817, 0.537692],
                 [0.119699, 0.618490, 0.536347],
                 [0.120081, 0.622161, 0.534946],
                 [0.120638, 0.625828, 0.533488],
                 [0.121380, 0.629492, 0.531973],
                 [0.122312, 0.633153, 0.530398],
                 [0.123444, 0.636809, 0.528763],
                 [0.124780, 0.640461, 0.527068],
                 [0.126326, 0.644107, 0.525311],
                 [0.128087, 0.647749, 0.523491],
                 [0.130067, 0.651384, 0.521608],
                 [0.132268, 0.655014, 0.519661],
                 [0.134692, 0.658636, 0.517649],
                 [0.137339, 0.662252, 0.515571],
                 [0.140210, 0.665859, 0.513427],
                 [0.143303, 0.669459, 0.511215],
                 [0.146616, 0.673050, 0.508936],
                 [0.150148, 0.676631, 0.506589],
                 [0.153894, 0.680203, 0.504172],
                 [0.157851, 0.683765, 0.501686],
                 [0.162016, 0.687316, 0.499129],
                 [0.166383, 0.690856, 0.496502],
                 [0.170948, 0.694384, 0.493803],
                 [0.175707, 0.697900, 0.491033],
                 [0.180653, 0.701402, 0.488189],
                 [0.185783, 0.704891, 0.485273],
                 [0.191090, 0.708366, 0.482284],
                 [0.196571, 0.711827, 0.479221],
                 [0.202219, 0.715272, 0.476084],
                 [0.208030, 0.718701, 0.472873],
                 [0.214000, 0.722114, 0.469588],
                 [0.220124, 0.725509, 0.466226],
                 [0.226397, 0.728888, 0.462789],
                 [0.232815, 0.732247, 0.459277],
                 [0.239374, 0.735588, 0.455688],
                 [0.246070, 0.738910, 0.452024],
                 [0.252899, 0.742211, 0.448284],
                 [0.259857, 0.745492, 0.444467],
                 [0.266941, 0.748751, 0.440573],
                 [0.274149, 0.751988, 0.436601],
                 [0.281477, 0.755203, 0.432552],
                 [0.288921, 0.758394, 0.428426],
                 [0.296479, 0.761561, 0.424223],
                 [0.304148, 0.764704, 0.419943],
                 [0.311925, 0.767822, 0.415586],
                 [0.319809, 0.770914, 0.411152],
                 [0.327796, 0.773980, 0.406640],
                 [0.335885, 0.777018, 0.402049],
                 [0.344074, 0.780029, 0.397381],
                 [0.352360, 0.783011, 0.392636],
                 [0.360741, 0.785964, 0.387814],
                 [0.369214, 0.788888, 0.382914],
                 [0.377779, 0.791781, 0.377939],
                 [0.386433, 0.794644, 0.372886],
                 [0.395174, 0.797475, 0.367757],
                 [0.404001, 0.800275, 0.362552],
                 [0.412913, 0.803041, 0.357269],
                 [0.421908, 0.805774, 0.351910],
                 [0.430983, 0.808473, 0.346476],
                 [0.440137, 0.811138, 0.340967],
                 [0.449368, 0.813768, 0.335384],
                 [0.458674, 0.816363, 0.329727],
                 [0.468053, 0.818921, 0.323998],
                 [0.477504, 0.821444, 0.318195],
                 [0.487026, 0.823929, 0.312321],
                 [0.496615, 0.826376, 0.306377],
                 [0.506271, 0.828786, 0.300362],
                 [0.515992, 0.831158, 0.294279],
                 [0.525776, 0.833491, 0.288127],
                 [0.535621, 0.835785, 0.281908],
                 [0.545524, 0.838039, 0.275626],
                 [0.555484, 0.840254, 0.269281],
                 [0.565498, 0.842430, 0.262877],
                 [0.575563, 0.844566, 0.256415],
                 [0.585678, 0.846661, 0.249897],
                 [0.595839, 0.848717, 0.243329],
                 [0.606045, 0.850733, 0.236712],
                 [0.616293, 0.852709, 0.230052],
                 [0.626579, 0.854645, 0.223353],
                 [0.636902, 0.856542, 0.216620],
                 [0.647257, 0.858400, 0.209861],
                 [0.657642, 0.860219, 0.203082],
                 [0.668054, 0.861999, 0.196293],
                 [0.678489, 0.863742, 0.189503],
                 [0.688944, 0.865448, 0.182725],
                 [0.699415, 0.867117, 0.175971],
                 [0.709898, 0.868751, 0.169257],
                 [0.720391, 0.870350, 0.162603],
                 [0.730889, 0.871916, 0.156029],
                 [0.741388, 0.873449, 0.149561],
                 [0.751884, 0.874951, 0.143228],
                 [0.762373, 0.876424, 0.137064],
                 [0.772852, 0.877868, 0.131109],
                 [0.783315, 0.879285, 0.125405],
                 [0.793760, 0.880678, 0.120005],
                 [0.804182, 0.882046, 0.114965],
                 [0.814576, 0.883393, 0.110347],
                 [0.824940, 0.884720, 0.106217],
                 [0.835270, 0.886029, 0.102646],
                 [0.845561, 0.887322, 0.099702],
                 [0.855810, 0.888601, 0.097452],
                 [0.866013, 0.889868, 0.095953],
                 [0.876168, 0.891125, 0.095250],
                 [0.886271, 0.892374, 0.095374],
                 [0.896320, 0.893616, 0.096335],
                 [0.906311, 0.894855, 0.098125],
                 [0.916242, 0.896091, 0.100717],
                 [0.926106, 0.897330, 0.104071],
                 [0.935904, 0.898570, 0.108131],
                 [0.945636, 0.899815, 0.112838],
                 [0.955300, 0.901065, 0.118128],
                 [0.964894, 0.902323, 0.123941],
                 [0.974417, 0.903590, 0.130215],
                 [0.983868, 0.904867, 0.136897],
                 [0.993248, 0.906157, 0.143936]]

from matplotlib.colors import ListedColormap

viridis = ListedColormap(_viridis_data, name='viridis')